# What's the Notebook for?

This competition has 11,898 bounding boxes and 4,919 annotated. It looks sufficient to train the model, but considering they are sourced from almost identical successive frames, it is doubtful that they are sufficient.

Can we inprove the detection model by adding new synthesized images in GAN? This is the first motivation of this approach.

Here, I used the model Pedestrian-Syntheis-GAN [1], which was used to generate pseudo-images of pedestrians. The model can be trained if we prepare an image in which the BBox and the target are replaced by noise, so I trained it using a partial dataset of the competition (about 1,200 sampled clips of size 256x256).

[1] https://arxiv.org/abs/1804.02047

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
ROOT_DIR = Path('/kaggle')
INPUT_DIR = ROOT_DIR / 'input'
WORK_DIR = ROOT_DIR / 'working'

train = pd.read_csv(INPUT_DIR / 'tensorflow-great-barrier-reef/train.csv')

In [ ]:
train['n_bbox'] = train['annotations'].apply(lambda x: len(eval(x)))

In [ ]:
# number of bounding boxes
train['n_bbox'].sum()

In [ ]:
# number of annotated frames
len(train.query('n_bbox > 0'))

In [ ]:
image_path = INPUT_DIR / 'starfish-generate-psgan-dataset/psgan_datasets/images/train/10.png'

# Input Image for Pedestrian-Synthesis GAN (PS-GAN)

In [ ]:
fig, ax = plt.subplots()
img = plt.imread(image_path)
ax.imshow(img)
plt.axis('off');

# Synthesized Results

Here, I can show part of the synthesized results.

In [ ]:
import os


def plot_real_and_fake_image(image_id,
                             root_path='/kaggle/input/starfish-psgan-results/results/img_128_v7/test_latest/images'):
    
    image_paths = [
        os.path.join(root_path, f'{image_id}_real_B.png'),
        os.path.join(root_path, f'{image_id}_real_A.png'),
        os.path.join(root_path, f'{image_id}_fake_B.png'),
    ]
    
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))
    
    labels = ['real', 'input', 'synthesized']
    for ax, image_path, label in zip(axs, image_paths, labels):
        img = plt.imread(image_path)
        ax.imshow(img)
        ax.set_title(label, fontsize=14)
        ax.axis('off')
        
    plt.tight_layout()

## Sample of Synthesized Images

In [ ]:
plot_real_and_fake_image(91)

In [ ]:
plot_real_and_fake_image(67)

In [ ]:
plot_real_and_fake_image(22)

In [ ]:
plot_real_and_fake_image(63)

I trained the model for about 150 epochs, but the latest model's discriminator seems overfitting because almost perfectly discriminates fake and real. So I used weight of 125 epoch.

The model sometimes generates COTS-like object, sometimes not. Since I fed only 1200 input data, I think the model are overfitted. It sometimes outputs completely fake-looking images when fed new image which doesn't apper in train dataset.

I doubt this model can generate the data useful for object detection.

## Sample of Poorly Sinthesized Images

In [ ]:
plot_real_and_fake_image(92)

In [ ]:
plot_real_and_fake_image(15)

# Reference

The original repository [2] has many bugs, so I forked and modified the repository [3]. 
(The modified repository might still contain many bugs.)

* [2] https://github.com/yueruchen/Pedestrian-Synthesis-GAN
* [3] https://github.com/bilzard/Pedestrian-Synthesis-GAN